In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pickle
import pandas as pd
import numpy as np
import os
from io import StringIO
import traceback

import tensorflow as tf
import keras

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler


2026-02-14 19:14:31.954238: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-14 19:14:32.861117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-14 19:14:36.331588: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
global model_lstm
global scaler_m

path_lstm = 'best_lstm.pkl'
path_scl = 'best_scaler.pkl'

model_lstm = pickle.load(open(path_lstm, 'rb'))
scaler_m = pickle.load(open(path_scl, 'rb'))


2026-02-14 19:14:37.805712: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (100)


In [4]:
def splitting_window(features, targets, n_steps, ages=None):
    """
    Splits the features, targets, and optionally ages into windows of size n_steps.
    Args:
        features (np.ndarray): Input features.
        targets (np.ndarray): Input targets.
        n_steps (int): Number of steps in each window.
        ages (np.ndarray, optional): Input ages. Defaults to None.

    Returns:
        np.ndarray: Windowed features.
        np.ndarray: Targets for each window.
        np.ndarray (optional): Windowed ages if ages is not None.
    """
    x_seq, y_seq, age_seq = [], [], []

    for i in range(len(features)):
        end_idx = i + n_steps
        if end_idx >= len(features):
            break
        x_seq.append(features[i:end_idx, :])
        y_seq.append(targets[end_idx])
        if ages is not None:
            age_seq.append(ages[end_idx])

    if ages is not None:
        return np.asarray(x_seq), np.asarray(y_seq), np.asarray(age_seq)
    else:
        return np.asarray(x_seq), np.asarray(y_seq)

def window_dataset_creation(n_steps, fixation, include_age):
    """
    Creates a windowed dataset from the input CSV file.

    Args:
        n_steps (int): Number of steps in each window.
        fixation (str): Data.
        include_age (int): Age.
    Returns:
        np.ndarray: Windowed features.
        np.ndarray: Windowed and one-hot encoded targets.
        np.ndarray: Windowed ages.
    """
    #Load the dataset
    data = pd.read_csv(fixation)

    df_id = data[data['SubjectID'] == data["SubjectID"].unique()[0]]
    df_id["Age"] = include_age
    #df_id["Group"] = -1
    y_train = df_id.loc[:, ["Group"]]

    age_train = df_id.loc[:, ["Age"]]
    x_train = df_id.loc[:, ["FIX_X", "FIX_Y", "FIX_DURATION"]]
    x_train_windowed_final, y_train_windowed_final, age_final = splitting_window(x_train.values, y_train.values, n_steps, age_train.values)
    
    for ids in data["SubjectID"].unique()[1:]:
        df_id = data[data['SubjectID'] == ids]
        y_train = df_id.loc[:, ["Group"]]

        age_train = df_id.loc[:, ["Age"]]
        x_train = df_id.loc[:, ["FIX_X", "FIX_Y", "FIX_DURATION"]]
        x_train_windowed, y_train_windowed, age_windowed = splitting_window(x_train.values, y_train.values, n_steps, age_train.values)
        age_final = np.concatenate((age_final, age_windowed))
        
        x_train_windowed_final = np.concatenate((x_train_windowed_final, x_train_windowed))
        y_train_windowed_final = np.concatenate((y_train_windowed_final, y_train_windowed))
    
    #One-hot encode the targets
    y_train_windowed_final = y_train_windowed_final.reshape(-1) - 1
    #print(y_train_windowed_final)
    y_train_windowed_final = tf.keras.utils.to_categorical(y_train_windowed_final)
    y_train_windowed_final = np.array(y_train_windowed_final)
    return x_train_windowed_final, y_train_windowed_final, age_final


In [5]:
#Программный компонент предоставляет пользователю возможность загрузки CSV файла, содержащего видеоокулографические данные, и возраст

In [6]:
#1.  Загрузить файл с данными видеоокулографии и возраст в модуль предварительной обработки данных;

In [7]:
file_path = 'test_case_108_10_cl1.csv' 
age_ = 10

file_content = ''

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
        
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

csvStringIO = StringIO(file_content)


n_steps = 10
X_test, y_test, age = window_dataset_creation(n_steps, csvStringIO, age_)
#print(age)

age_test = scaler_m.transform(age)
X_test_data = np.hstack((model_lstm.predict(X_test), age_test))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step


In [8]:
#2. Проверить, что модуль предварительной обработки не выдает ошибок, вызванных неверной реализацией.

In [9]:
print(X_test)
print('\n')
print(age_test)

[[[ 202.3  509.    24. ]
  [ 199.8  505.8  100. ]
  [ 197.8  506.3   77. ]
  [ 326.1  532.3  127. ]
  [ 323.4  535.3   42. ]
  [ 356.   537.9   43. ]
  [ 282.7  545.9   26. ]
  [ 287.   548.6  157. ]
  [ 477.3  565.4  102. ]
  [ 404.1  548.6  134. ]]

 [[ 199.8  505.8  100. ]
  [ 197.8  506.3   77. ]
  [ 326.1  532.3  127. ]
  [ 323.4  535.3   42. ]
  [ 356.   537.9   43. ]
  [ 282.7  545.9   26. ]
  [ 287.   548.6  157. ]
  [ 477.3  565.4  102. ]
  [ 404.1  548.6  134. ]
  [ 549.1  540.3   39. ]]

 [[ 197.8  506.3   77. ]
  [ 326.1  532.3  127. ]
  [ 323.4  535.3   42. ]
  [ 356.   537.9   43. ]
  [ 282.7  545.9   26. ]
  [ 287.   548.6  157. ]
  [ 477.3  565.4  102. ]
  [ 404.1  548.6  134. ]
  [ 549.1  540.3   39. ]
  [ 553.5  545.3   45. ]]

 [[ 326.1  532.3  127. ]
  [ 323.4  535.3   42. ]
  [ 356.   537.9   43. ]
  [ 282.7  545.9   26. ]
  [ 287.   548.6  157. ]
  [ 477.3  565.4  102. ]
  [ 404.1  548.6  134. ]
  [ 549.1  540.3   39. ]
  [ 553.5  545.3   45. ]
  [ 610.2  547.3  1

In [10]:
#Программный компонент выдает категориальную оценку наличия признаков дислексии после обработки загруженных данных

In [11]:
#1. Инициализировать модель

In [12]:
path_mdl = 'best_model.pkl'
model_best = pickle.load(open(path_mdl, 'rb'))

In [13]:
#2. Передать в модель предварительно обработанный набор данных;
#3. Запустить обработку данных моделью;

In [14]:
res1 = model_best.predict(X_test_data)

In [15]:
#4. Проверить, что модель выдает результат классификации.

In [16]:
print('\nОценка категории для каждой фиксации')
print(res1+1)

values, counts = np.unique(res1, return_counts=True)
ind = np.argmax(counts)
res = values[ind]

print('\nПолученная итоговая оценка категории испытуемого: ' + str(res + 1) + ' (норма)')



Оценка категории для каждой фиксации
[1 1 1 1 1 1 1 1 1 1 1 1]

Полученная итоговая оценка категории испытуемого: 1 (норма)


In [17]:
#Общее время обработки данных (от момента завершения загрузки входных данных до получения результата анализа) должно составлять не 
#более 30 секунд для данных одного участника (испытуемого)

In [18]:
import time

start_time = time.perf_counter()

file_path = 'test_case_s161_30_cl2.csv' 
age_ = 9

file_content = ''

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
        
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

csvStringIO = StringIO(file_content)

n_steps = 10
X_test, y_test, age = window_dataset_creation(n_steps, csvStringIO, age_)

age_test = scaler_m.transform(age)
X_test_data = np.hstack((model_lstm.predict(X_test), age_test))

res2 = model_best.predict(X_test_data)

print('\nОценка категории для каждой фиксации')
print(res2+1)

values, counts = np.unique(res2, return_counts=True)
ind = np.argmax(counts)
res = values[ind]


print('\nПолученная итоговая оценка категории испытуемого: ' + str(res + 1) + ' (дислексия)')

end_time = time.perf_counter()


elapsed_time = end_time - start_time
print(f"Общее время выполнения: {elapsed_time:.4f} секунд")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step 

Оценка категории для каждой фиксации
[2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]

Полученная итоговая оценка категории испытуемого: 2 (дислексия)
Общее время выполнения: 0.5185 секунд
